<div style="color:red;background-color:black">
Diamond Light Source
<br style="color:red;background-color:antiquewhite"><h1>Python Libraries: SciKitLearn</h1>  

©2000-20 Chris Seddon 
</div>

## 1
The iris flower is an excellent starting point for studying machine learning.  There are 3 common species of iris: setosa, versicolor and virginica.  What is interesting about these species is that they are easy to classify if we measure their petal and sepal width and length.  Each species cluster around a region in 4 dimensional space (the dimensions being (petal width, petal length, sepal width, sepal length).

Let's start with a picture:

In [ ]:
from PIL import Image

image = Image.open("data/images/iris.png")
image

## 2
Let's examine our dataset with Pandas:

In [ ]:
import pandas as pd
pd.set_option('display.width', None)        # None means all data displayed


# load iris data set
iris_df = pd.read_csv("data/iris.csv")
print(iris_df)

## 3
It will be helpful to add a new column to our dataframe where we use integers for the species.  We'll call this column 'target':

In [ ]:
import pandas as pd
pd.set_option('display.width', None)        # None means all data displayed


# load iris data set
iris_df = pd.read_csv("data/iris.csv")
target_names = {'setosa':0, 'versicolor':1, 'virginica':2}
iris_df["target"] = iris_df.apply(lambda row: target_names[row.species], axis=1, raw=True)
print(iris_df.sample(10))

## 4
With machine learning, we use the 4 dimensional space of (sepal_length, sepal_width, petal_length and petal_width) to classify the irises.  To get a feel for this, we would like to plot the 4 dimensional space with each iris, but that's impossible.  However, what we can do is create four 3 dimensional cross sections of the 4 dimensional space:

In [ ]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import pandas as pd

colors = ["red", "green", "blue", "black", "black"]
markers = ["o", "o", "o", "D", "D"]
sizes = [10, 10, 10, 100, 100]
labels = ['sepal length', 'sepal width', 'petal length', 'petal width']
target_names = {'setosa':0, 'versicolor':1, 'virginica':2}

iris_df = pd.read_csv("data/iris.csv")
iris_df["target"] = iris_df.apply(lambda row: target_names[row.species], axis=1, raw=True)
iris_df.drop(["species"], axis = 1, inplace = True)

# plot
figure = plt.figure(figsize=(16, 12))
        
def scatter(subplot, i, j, k):
    def doit(row):
        n = int(row[4])          
        ax.scatter(row[i], row[j], row[k],   
                   c=colors[n], 
                   marker=markers[n],
                   s=sizes[n])

    ax = figure.add_subplot(subplot, projection='3d')
    ax.set_xlabel(labels[i])
    ax.set_ylabel(labels[j])
    ax.set_zlabel(labels[k])
    iris_df.apply(doit, axis=1, raw=True)

scatter(221, 0, 1, 2)
scatter(222, 0, 1, 3)
scatter(223, 0, 2, 3)
scatter(224, 1, 2, 3)
plt.show()

## 5
Now we take our dataframe and use it to train our model.  There are several different model we can use.  We will be using:<pre>KNeighborsClassifier
LogisticRegression</pre>

Training the model involves separation the 4 "parameters" (sepal_length, sepal_width, petal_length and petal_width) from the "target" column in our dataframe and passing these two dataframes to an estimator.  In our case, the estimator will have 150 rows to work with.

Next we introduce 3 new irises that haven't been classified yet.  Since each model has been trained, it can now predict which type of iris they think they are.

Note the use of "iloc" to extract columns from the dataframe.

In [ ]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

target_names = {'setosa':0, 'versicolor':1, 'virginica':2}
iris_df = pd.read_csv("data/iris.csv")
iris_df["target"] = iris_df.apply(lambda row: target_names[row.species], axis=1, raw=True)
iris_df.drop(["species"], axis = 1, inplace = True)

# create 2 new dataframes to pass to the estimator
parameters = iris_df.iloc[:,[0,1,2,3]]
target = iris_df.iloc[:,4]

# define three unclassified iris's
iris1 = [4.1, 3.1, 1.8, 0.5]
iris2 = [6.9, 3.5, 3.5, 2.5]
iris3 = [6.7, 2.0, 5.0, 1.6] 

def predict(estimator, message):
    estimator.fit(parameters, target)     # train
    print(message, estimator.predict([iris1, iris2, iris3])) # estimate

# predict with different estimators and parameters
predict(KNeighborsClassifier(n_neighbors=1), "KNeighbors(K=1):")
predict(KNeighborsClassifier(n_neighbors=3), "KNeighbors(K=3):")
predict(KNeighborsClassifier(n_neighbors=5), "KNeighbors(K=5):")
predict(LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=150), "LogisticRegression:")

## 6
As you can see, there is some disagreement over what species the third iris belongs to.  

Let's look at our 4 plots again, this time with the 3 new iris plotted.  Our 3 new irises are colored black, grey and silver:

In [ ]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import pandas as pd

colors = ["red", "green", "blue", "black", "grey", "silver"]
markers = ["o", "o", "o", "D", "D", "D"]
sizes = [10, 10, 10, 100, 100, 100]
labels = ['sepal length', 'sepal width', 'petal length', 'petal width']
target_names = {'setosa':0, 'versicolor':1, 'virginica':2}

iris_df = pd.read_csv("data/iris.csv")
iris_df["target"] = iris_df.apply(lambda row: target_names[row.species], axis=1, raw=True)
iris_df.drop(["species"], axis = 1, inplace = True)
iris1 = [4.1, 3.1, 1.8, 0.5, 3]
iris2 = [6.9, 3.5, 2.5, 2.5, 4]  
iris3 = [6.7, 3.0, 5.2, 2.3, 5] 
df = pd.DataFrame([iris1, iris2, iris3], columns = iris_df.columns) 
iris_df = iris_df.append(df)

# plot
figure = plt.figure(figsize=(16, 12))
def scatter(subplot, i, j, k):
    def doit(row):
        n = int(row[4])          
        ax.scatter(row[i], row[j], row[k],   
                   c=colors[n], 
                   marker=markers[n],
                   s=sizes[n])

    ax = figure.add_subplot(subplot, projection='3d')
    ax.set_xlabel(labels[i])
    ax.set_ylabel(labels[j])
    ax.set_zlabel(labels[k])
    iris_df.apply(doit, axis=1, raw=True)

scatter(221, 0, 1, 2)
scatter(222, 0, 1, 3)
scatter(223, 0, 2, 3)
scatter(224, 1, 2, 3)
plt.show()

## 7
Bokeh is a Python interactive visualization library that is an alternative to using MatPlotLib.  Bokeh is designed to targets modern web browsers, but does work inline with Jupyter notebook provide you insert the line"<pre>
from bokeh.io import output_notebook
output_notebook()</pre>

Here we use Bokeh to visualize our original Iris dataset:

In [ ]:
import pandas as pd
from bokeh.plotting import figure, show, output_file
from bokeh.transform import factor_cmap, factor_mark
from bokeh.io import output_notebook

output_notebook()

iris_df = pd.read_csv("data/iris.csv")
flowers = iris_df
SPECIES = ['setosa', 'versicolor', 'virginica']
MARKERS = ['hex', 'circle_x', 'triangle']

p = figure(title = "Iris Morphology")
p.xaxis.axis_label = 'Petal Length'
p.yaxis.axis_label = 'Sepal Width'

p.scatter("petal_length", "sepal_width", 
          source=iris_df, 
          fill_alpha=0.4, 
          size=12,
          marker=factor_mark('species', MARKERS, SPECIES),
          color=factor_cmap('species', 'Category10_3', SPECIES))
show(p)

## 8
Seaborn is another useful plotting library.  It is built on top of matplotlib and closely integrated with pandas data structures.  This is our original iris dataset with Seaborn:

In [ ]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)

sns.set(style="whitegrid", palette="muted")

# Load the iris dataset
iris_df = pd.read_csv("data/iris.csv")

# "Melt" the dataset
iris_df = pd.melt(iris_df, id_vars="species", 
                  var_name='measurement', value_name='cm') 

# Draw a categorical scatterplot to show each observation
sns.swarmplot(x="measurement", 
              y="cm", 
              hue="species",
              palette=["r", "c", "y"], 
              data=iris_df)
plt.show()

## 9
In the above code we use Pandas "melt" method.  This is essentially a pivot function.  The original datafile is pivoted on "species"; this is the "id_vars" parameter.  All other columns are pivoted on this column.

The resulting dataframe has 3 columns:
* species: the pivot column
* measurement: the name of one of the other columns
* cm: the value of this column

and a sample set of rows looks like:

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)

sns.set(style="whitegrid", palette="muted")

# Load the iris dataset
iris_df = pd.read_csv("data/iris.csv")

# "Melt" the dataset
iris_df = pd.melt(iris_df, id_vars="species", 
                  var_name='measurement', value_name='cm') 
print(iris_df.sample(20))